In [1]:
import numpy as np
import pandas as pd

CONVERSION_FACTOR = 1e-6 / .0023

# maps to rename timepoints, knocked-out genes, and reactions
TIME_MAP = { f'J{i+1:02}': i+1 for i in range(4) }
TIME_MAP['EP'] = 11
KO_MAP = { 'gnd': 'GND', 'pgi': 'PGI', 'ptsHIcrr': 'GLCptspp', 'sdhCB': 'SUCDi', 'tpiA': 'TPI' }
RATE_MAP = { 'biomass': 'Ec_biomass_iJO1366_WT_53p95M', 'glc-D': 'DM_glc_e', 'ac': 'DM_ac_e', 'succ': 'DM_succ_e', 'lac-D': 'DM_lac-D_e' }
# regex pattern used to extract relevant fields from the sample short name
NAME_PATTERN = r'OxicEvo04(?P<ko>tpiA|pgi|sdhCB|gnd|ptsHIcrr)?(?:Evo(?P<evo>\d+))?(?P<time>J\d+|EP)?Ecoli(?P<media>Glc\w*)_Broth-(?P<replicate>\d+)'

# Log Concentration Bounds

1. Read data, trim unused rows, and format labels

In [2]:
met_data = pd.read_csv('data/data_stage02_quantification_dataPreProcessing_replicates.csv', index_col = 0)

# keep only concentration rows, remove 2nd fragments
mask = ( met_data['calculated_concentration_units'] == 'umol*gDW-1' ) & ~( met_data['component_name'].str.contains('_2.Light') )
met_data = met_data[mask]

# extract KO gene, evo number, replicate, and media
pattern = r'OxicEvo04(?P<ko>tpiA|pgi|sdhCB|gnd|ptsHIcrr)?(?:Evo(?P<evo>\d+))?(?P<time>J\d+|EP)?Ecoli(?P<media>Glc\w*)_Broth-(?P<replicate>\d+)'
extra_cols = met_data['sample_name_short'].str.extract( pattern )

met_data = pd.concat( [ met_data, extra_cols ], axis = 1 )

met_data.head()

,analysis_id,experiment_id,sample_name_short,time_point,component_group_name,component_name,imputation_method,calculated_concentration,calculated_concentration_units,used_,comment_,ko,evo,time,media,replicate
id,,,,,,,,,,,,,,,,
4706751,ALEsKOs01_Metabolomics_0-1-2-11_evo04pgiEvo01,ALEsKOs01,OxicEvo04pgiEvo01EPEcoliGlc_Broth-2,11,lac-D,lac-D.lac-D_1.Light,ameliaII,1.109854,umol*gDW-1,t,NaN,pgi,01,EP,Glc,2
4706752,ALEsKOs01_Metabolomics_0-1-2-11_evo04pgiEvo01,ALEsKOs01,OxicEvo04pgiEvo01J02EcoliGlc_Broth-2,2,dimp,dimp.dimp_1.Light,ameliaII,0.009522,umol*gDW-1,t,NaN,pgi,01,J02,Glc,2
4707023,ALEsKOs01_Metabolomics_0-1-2-11_evo04pgiEvo01,ALEsKOs01,OxicEvo04pgiEvo01J01EcoliGlc_Broth-5,1,dimp,dimp.dimp_1.Light,ameliaII,0.004451,umol*gDW-1,t,NaN,pgi,01,J01,Glc,5
4471944,ALEsKOs01_Metabolomics_0-1-2-11_evo04pgiEvo01,ALEsKOs01,OxicEvo04pgiEcoliGlc_Broth-1,0,23dpg,23dpg.23dpg_1.Light,NaN,0.200755,umol*gDW-1,t,NaN,pgi,NaN,NaN,Glc,1
4471945,ALEsKOs01_Metabolomics_0-1-2-11_evo04pgiEvo01,ALEsKOs01,OxicEvo04pgiEcoliGlc_Broth-1,0,6pgc,6pgc.6pgc_1.Light,NaN,7.065421,umol*gDW-1,t,NaN,pgi,NaN,NaN,Glc,1


2. Continue formatting data

In [3]:
# map time points
met_data['time'] = met_data['time'].map(TIME_MAP).fillna(0).astype(int)

assert sum( met_data['time'] == met_data['time_point'] ) == len(met_data)

# map KO gene to reaction ID in model
met_data['ko'] = met_data['ko'].map(KO_MAP).fillna('wt')
met_data['strain'] = met_data.pop('evo').fillna(0).astype(int)

cols = [ 'strain', 'ko', 'media', 'time_point', 'component_group_name', 'calculated_concentration' ]

met_data[ [ *cols, 'replicate' ] ].head()

,strain,ko,media,time_point,component_group_name,calculated_concentration,replicate
id,,,,,,,
4706751,1,PGI,Glc,11,lac-D,1.109854,2
4706752,1,PGI,Glc,2,dimp,0.009522,2
4707023,1,PGI,Glc,1,dimp,0.004451,5
4471944,0,PGI,Glc,0,23dpg,0.200755,1
4471945,0,PGI,Glc,0,6pgc,7.065421,1


3. Find min/max concentration and create output file

In [4]:
# group data and calculate min/max
met_group = met_data[ cols ].groupby( cols[ :-1 ] )
lc_bounds = met_group.agg( [ ('min', np.min), ('max', np.max) ] ).reset_index()

# rename columns, convert concentrations (μmol/gCDW) to log concentrations (M)
lc_bounds.columns = [ *cols[:-2], 'id', 'lb', 'ub' ]
lc_bounds.iloc[:, -2:] = np.log( lc_bounds.iloc[:, -2:] * CONVERSION_FACTOR )

lc_bounds.head()

,strain,ko,media,time_point,id,lb,ub
0,0,GLCptspp,Glc,0,23dpg,-7.860059,-7.424463
1,0,GLCptspp,Glc,0,6pgc,-8.785181,-8.248013
2,0,GLCptspp,Glc,0,Lcystin,-9.206868,-8.559319
3,0,GLCptspp,Glc,0,Pool_2pg_3pg,-6.367578,-5.712819
4,0,GLCptspp,Glc,0,accoa,-8.114400,-7.619540


# Reaction Bounds

1. Read physiological data and repeat processing steps.

In [5]:
rate_data = pd.read_csv('data/ALEsKOs01_uptake-secretion-growth_rates.csv', index_col = 0)

extra_cols = rate_data['sample_name_short'].str.extract(NAME_PATTERN)

rate_data = pd.concat( [ rate_data, extra_cols ], axis = 1 )

# map time points, KO genes, reactions, and strain #
rate_data['time_point'] = rate_data.pop('time').map(TIME_MAP).fillna(0).astype(int)
rate_data['ko'] = rate_data['ko'].map(KO_MAP).fillna('wt')
rate_data['strain'] = rate_data.pop('evo').fillna(0).astype(int)
rate_data['reaction'] = rate_data['met_id'].map(RATE_MAP)

rate_data.head()

,experiment_id,sample_name_short,met_id,slope,intercept,r2,rate,rate_units,p_value,std_err,used_,comment_,ko,media,replicate,time_point,strain,reaction
id,,,,,,,,,,,,,,,,,,
75,ALEsKOs01,OxicEvo04pgiEcoliGlc_Broth-1,biomass,0.169432,-2.993065e+06,0.988603,0.169432,hr-1,4.681715e-10,0.006064,True,NaN,PGI,Glc,1,0,0,Ec_biomass_iJO1366_WT_53p95M
76,ALEsKOs01,OxicEvo04pgiEcoliGlc_Broth-2,biomass,0.186814,-3.300124e+06,0.998051,0.186814,hr-1,5.699384e-08,0.003692,True,NaN,PGI,Glc,2,0,0,Ec_biomass_iJO1366_WT_53p95M
77,ALEsKOs01,OxicEvo04pgiEcoliGlc_Broth-3,biomass,0.166212,-2.936195e+06,0.996168,0.166212,hr-1,3.090230e-07,0.004610,True,NaN,PGI,Glc,3,0,0,Ec_biomass_iJO1366_WT_53p95M
78,ALEsKOs01,OxicEvo04ptsHIcrrEcoliGlc_Broth-1,biomass,0.183147,-3.235488e+06,0.914149,0.183147,hr-1,4.270530e-06,0.018709,True,NaN,GLCptspp,Glc,1,0,0,Ec_biomass_iJO1366_WT_53p95M
79,ALEsKOs01,OxicEvo04ptsHIcrrEcoliGlc_Broth-2,biomass,0.196312,-3.468061e+06,0.937268,0.196312,hr-1,1.030253e-06,0.016929,True,NaN,GLCptspp,Glc,2,0,0,Ec_biomass_iJO1366_WT_53p95M


2. Find min/max rate per condition and create output file.

In [7]:
cols = [ 'strain', 'ko', 'media', 'time_point', 'reaction', 'rate' ]

# drop yields (i.e., rows with NaN values in one of the specified columns)
rate_data.drop( index = rate_data[ rate_data[ cols ].isnull().any(axis = 1) ].index, inplace = True )

# group data and calculate min/max
rate_group = rate_data[ cols ].groupby( cols[ :-1 ] )
rate_bounds = rate_group.agg( [ ('min', np.min), ('max', np.max) ] ).reset_index()
rate_bounds.columns = [ *cols[:-2], 'id', 'lb', 'ub' ]

rate_bounds.head()

,strain,ko,media,time_point,id,lb,ub
0,0,GLCptspp,Glc,0,DM_ac_e,0.119984,0.119984
1,0,GLCptspp,Glc,0,DM_glc_e,-5.005564,-3.781214
2,0,GLCptspp,Glc,0,Ec_biomass_iJO1366_WT_53p95M,0.183147,0.200345
3,0,GND,Glc,0,DM_ac_e,10.299573,19.892670
4,0,GND,Glc,0,DM_glc_e,-23.666876,-12.929865


3. Patch missing bounds:
    
    - Add 0 bounds to knocked-out reactions

    - Only time points 0 and 11 have measurements for uptake/secretion rates. To constrain the middle points, I determine the min/max values between the known time points. For example,

<table>
    <tr>
        <th>strain</th>
        <th>time_point</th>
        <th>reaction</th>
        <th>lb</th>
        <th>ub</th>
    </tr>
    <tr>
        <td>Ecoli ΔPGI</td>
        <td>0</td>
        <td>glucose exchange</td>
        <td>-3.0</td>
        <td><b>-2.0</b></td>
    </tr>
    <tr>
        <td>Ecoli ΔPGI</td>
        <td>11</td>
        <td>glucose exchange</td>
        <td><b>-7.0</b></td>
        <td>-6.0</td>
    </tr>
</table>

becomes:

<table>
    <tr>
        <th>strain</th>
        <th>time_point</th>
        <th>reaction</th>
        <th>lb</th>
        <th>ub</th>
    </tr>
    <tr>
        <td>Ecoli ΔPGI</td>
        <td>0</td>
        <td>glucose exchange</td>
        <td>-3.0</td>
        <td>-2.0</td>
    </tr>
    <tr>
        <td>Ecoli ΔPGI</td>
        <td>1</td>
        <td>glucose exchange</td>
        <td><b>-7.0</b></td>
        <td><b>-2.0</b></td>
    </tr>
    <tr>
        <td>Ecoli ΔPGI</td>
        <td>3</td>
        <td>glucose exchange</td>
        <td><b>-7.0</b></td>
        <td><b>-2.0</b></td>
    </tr>
    <tr>
        <td>Ecoli ΔPGI</td>
        <td>11</td>
        <td>glucose exchange</td>
        <td>-7.0</td>
        <td>-6.0</td>
    </tr>
</table>

This may not be necessary, but the idea here is that the evolving strains should not secrete or consume more than at t = { 0, 11 }.

In [8]:
exchanges = [ *RATE_MAP.values() ][1:]

rows = []

t0_mask = rate_bounds['time_point'] == 0
tf_mask = rate_bounds['time_point'] == 11

for ko_id in rate_bounds['ko'].unique():
    ko_mask = rate_bounds['ko'] == ko_id
    for media in rate_bounds[ ko_mask ]['media'].unique():
        media_mask = rate_bounds['media'] == media
        for strain in rate_bounds[ ko_mask & media_mask ]['strain'].unique():
            strain_mask = rate_bounds['strain'] == strain
            for time in rate_bounds[ ko_mask & media_mask & strain_mask ]['time_point'].unique():
                # create
                cols = dict( strain = strain, ko = ko_id, media = media, time_point = time, id = ko_id, lb = 0, ub = 0 )
                if ko_id != 'wt':
                    rows.append(cols)
                # only patch time points between 0 and 11
                if 0 < time < 11:
                    for reaction in exchanges:
                        rxn_mask = rate_bounds['id'] == reaction
                        bounds = rate_bounds[ ko_mask & media_mask & rxn_mask & ( t0_mask | ( strain_mask & tf_mask ) ) ][ ['lb', 'ub'] ].values
                        # make sure there are measurements for this exchange/reaction
                        if bounds.shape[0] > 0:
                            row = cols.copy()
                            row['id'] = reaction
                            # in some cases, a metabolite (e.g., lactate) is only secreted at t = 11,
                            # add a 0 to represent secretion at t = 0
                            # most likely, the 0 will be picked as lower bound by the min function
                            if bounds.shape[0] == 1:
                                bounds = np.append(bounds, 0)
                            row['lb'] = bounds.min()
                            row['ub'] = bounds.max()
                            rows.append(row)

rate_bounds = pd.concat([rate_bounds, pd.DataFrame(rows)], ignore_index = True)

rate_bounds.tail()

,strain,ko,media,time_point,id,lb,ub
302,4,TPI,Glc,3,TPI,0.000000,0.000000
303,4,TPI,Glc,3,DM_glc_e,-10.608269,-2.804622
304,4,TPI,Glc,3,DM_ac_e,0.767761,2.966747
305,4,TPI,Glc,3,DM_lac-D_e,0.000000,3.682118
306,4,TPI,Glc,11,TPI,0.000000,0.000000


# All Together Now

Combine into a single DataFrame.

In [9]:
lc_bounds.insert( 0, 'bound_type', 'log_concentration' )
rate_bounds.insert( 0, 'bound_type', 'reaction' )

bounds = pd.concat([lc_bounds, rate_bounds], ignore_index = True)
bounds.to_csv('data/bounds.csv', index = False)

bounds.head()

,bound_type,strain,ko,media,time_point,id,lb,ub
0,log_concentration,0,GLCptspp,Glc,0,23dpg,-7.860059,-7.424463
1,log_concentration,0,GLCptspp,Glc,0,6pgc,-8.785181,-8.248013
2,log_concentration,0,GLCptspp,Glc,0,Lcystin,-9.206868,-8.559319
3,log_concentration,0,GLCptspp,Glc,0,Pool_2pg_3pg,-6.367578,-5.712819
4,log_concentration,0,GLCptspp,Glc,0,accoa,-8.114400,-7.619540
